In [ ]:
import os
import torch
from torch import nn
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision import transforms as T
from sklearn.model_selection import KFold

In [ ]:

k_folds = 5
num_epochs = 25
loss_function = nn.CrossEntropyLoss()

In [ ]:

results = {}
torch.manual_seed(42)

image_folder_path = "C:/Users/Sahilur Rahman/OneDrive/Desktop/Tumor_Cellularity/20X_ROIs/"  
transform = T.Compose([
        T.RandomHorizontalFlip(),
        T.RandomVerticalFlip(),
        T.RandomApply(torch.nn.ModuleList([T.ColorJitter()]), p=0.20),
        T.RandomRotation(degrees=(-45, 45)),
        T.RandomAffine(degrees=0, translate=(0.2, 0.2), scale=(0.8, 1.2), shear=(-10, 10)),
        T.Resize(256),
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = ImageFolder(root=image_folder_path, transform=transform)

In [ ]:
kfold = KFold(n_splits=k_folds, shuffle=True)
model = torch.hub.load('facebookresearch/deit:main', 'deit_base_patch16_224', pretrained=True)

for param in model.parameters():
    param.requires_grad = False
n_inputs = model.head.in_features
model.head = nn.Sequential(
    nn.Linear(n_inputs, 2048),
    nn.BatchNorm1d(2048),
    nn.ReLU(inplace=True),
    nn.Dropout(0.5),
    nn.Linear(2048, 1024),
    nn.BatchNorm1d(1024),
    nn.ReLU(inplace=True),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(inplace=True),
    nn.Dropout(0.5),
    nn.Linear(512, len(classes))
)
model = model.to(device)

In [ ]:

print('--------------------------------')
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    print(f'FOLD {fold}')
    print('--------------------------------')
    train_subsampler = SubsetRandomSampler(train_ids)
    test_subsampler = SubsetRandomSampler(test_ids)
    trainloader = DataLoader(dataset, batch_size=128, sampler=train_subsampler)
    testloader = DataLoader(dataset, batch_size=128, sampler=test_subsampler)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    
    for epoch in range(0, num_epochs):
      
        print(f'Starting epoch {epoch + 1}')
        current_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, targets = data
            inputs, targets = inputs.to('cuda'), targets.to('cuda')
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_function(outputs, targets)
            loss.backward()
            optimizer.step()
            current_loss += loss.item()
            if i % 500 == 499:
                print('Loss after mini-batch %5d: %.3f' % (i + 1, current_loss / 500))
                current_loss = 0.0
    print('Training process has finished. Saving trained model.')
    print('Starting testing')

In [ ]:
correct, total = 0, 0
predictions = []
targets_list = []
with torch.no_grad():
        for i, data in enumerate(testloader, 0):
            inputs, targets = data
            inputs, targets = inputs.to('cuda'), targets.to('cuda')
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
            predictions.extend(predicted.cpu().numpy())
            targets_list.extend(targets.cpu().numpy())
        precision = precision_score(targets_list, predictions, average='macro')
        recall = recall_score(targets_list, predictions, average='macro')
        f1 = f1_score(targets_list, predictions, average='macro')

In [ ]:

print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
print('Precision for fold %d: %.3f' % (fold, precision))
print('Recall for fold %d: %.3f' % (fold, recall))
print('F1-score for fold %d: %.3f' % (fold, f1))
print('--------------------------------')

In [ ]:

results['accuracy'].append(100.0 * (correct / total))
results['precision'].append(precision)
results['recall'].append(recall)
results['f1'].append(f1)

In [ ]:

print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
for metric, values in results.items():
    avg_metric = np.mean(values)
    std_metric = np.std(values)
    print(f'Average {metric}: {avg_metric:.3f} ± {std_metric:.3f}')